In [1]:
import gensim.summarization.bm25 
import gensim
import os
import numpy as np
import pickle
import spacy
import torch
import transformers
import pickle
import numpy as np
from nltk import SnowballStemmer

In [89]:
import bs4
import urllib.request
import re
import timeit
import os
import spacy

class WebsiteData():
    
    def __init__(self, website,depth, pathName):#,website,parser = 'lxml'):
        '''
        Constructor of the class. We let the user choose the parser type, but keep it default to the fastest parser.
        We are keeping the bsObject private because the end user should NOT get access to any HTML information.
        '''
        self.website = website
        self.pageDict = {}
        self.depth = depth
        self.pathName = pathName
        '''
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
        except:
            print("Website not accessible : " ,self.website)
            self.__bsObject = None
            return None
        '''
            
    def parseBaseWebsite(self, parser = 'lxml'):
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
            return 1
        except Exception as e:
            print("Website not accessible. Exception :  " ,e)
            self.__bsObject = None
            return None
    
    def __getBaseURL(self):
        '''
        Return the base url of the website being parsed
        '''
        count = 0
        base1 = []
        base = ''
        if re.findall('http[s]?', self.website):
            for i in self.website:
                if i == '/':
                    count= count+1
                if count == 3:
                    break
                base1.append(i)
        
            base = base.join(base1)
        
        return base
    
    def displayHtml(self):
        '''
        Display website in HTML Structure 
        '''
        print(self.__bsObject.prettify())
    
    def retrieveHrefs(self):
        '''
        Retrieve links from the HREF Tags in the webpage 
        '''
        self.__linksList = []
        baseUrl = self.__getBaseURL()
        print("base url : ", baseUrl)
        
       
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        
        
        for link in bodyContent.find_all('a'):
            #print(link)
            suffixURL = link.get('href')
            #print(type(suffixURL))
            print(suffixURL)
            
            if (suffixURL is not None) and ('www.investopedia.com' in suffixURL) and (suffixURL.endswith('.asp'))  :
                self.__linksList.append(str(suffixURL))
                
         
        return self.__linksList
    
    def saveWebsiteData(self, levels):
        '''
        Save the data from the website in a txt file. The method creates a file with the topic of the link as the file name.
        The code will be extended to store the data from all the links in the linksList in the future.
        '''
        pathName = self.pathName
        if levels == 0 :
            pathName = pathName
        else:
            for counter in range(levels):
                pathName = pathName +  "/level" + str(counter+1 )
                
        if not os.path.exists(os.path.join(os.getcwd(),pathName)):
            os.mkdir(os.path.join(os.getcwd(),pathName ))
                

        #headerContent = self.__bsObject.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        paragraphs = bodyContent.find_all('p')
        #headers = headerContent.find_all('')

        print("writing data")        
        file_name = os.path.join(os.path.join(os.getcwd(), pathName), (self.website).split("/")[-1] + ".txt")
        print(file_name)
        with open(file_name ,"w+", encoding='utf-8') as f:
            maxParaLen = 0
            sumParaLen = 0
            nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
            totalParaLen = len(paragraphs)
            for paragraph in paragraphs:
                print("paragraph   :  ", paragraph)
                if paragraph.find_parent('div').find_previous('h3') is not None:
                    heading = paragraph.find_parent('div').find_previous('h3')
                    if heading.find('span') is not None:
                        
                        heading_text = heading.find('span').text
                    else:
                        heading_text = heading.text
                else:
                    heading = paragraph.find_parent('div').find_previous('h1')
                    heading_text = heading.text
                    
                 
                paraText = heading_text+ ". " + paragraph.getText()
                paraTextDoc = nlp(paraText)
                paraTextDocList = [tokens.text for tokens in paraTextDoc if tokens.text.strip() != '']
                paraLen = len(paraTextDocList)
                paraText = " ".join(paraTextDocList)
                sumParaLen = sumParaLen + paraLen
                if paraLen > maxParaLen:
                    maxParaLen = paraLen

                f.write(paraText + '\n--------------------------\n')
                

            avgParaLen = (sumParaLen/totalParaLen)
            self.pageDict[(self.website).split("/")[-1]] = (maxParaLen , avgParaLen)

            f.close()
            return self.pageDict
        


class WebsiteParser:
    
    def __init__(self,baseWebsite,level, pathName):
        
        self.baseWebsite = baseWebsite
        self.level = level
        self.pathName = pathName
    
    def parseWebsite(self):
        
        pageDict ={}
        linkDict = {}
        
        for level in range(self.level):
            
            if level == 0:
                obj = WebsiteData(self.baseWebsite,level, self.pathName)
                objret = obj.parseBaseWebsite()
                linkDict[level +1] = obj.retrieveHrefs()
                if objret is not None: 
                    data = obj.saveWebsiteData(level) 
                    pageDict.update(data)
            else:
                linkDict[level +1] = []
        
                if level < (self.level - 1):
                
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                       
                        linkDict[level +1].extend(obj1.retrieveHrefs())
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)
                
                else:
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)

In [90]:
numLevels = 1
initLink = 'https://www.investopedia.com/terms/r/riskmanagement.asp'
path = r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data"
obj = WebsiteParser(initLink,numLevels,path)
print(obj.parseWebsite())

base url :  https://www.investopedia.com
https://www.investopedia.com/terms/f/financialplanner.asp
https://www.investopedia.com/terms/r/risk-analysis.asp
https://www.investopedia.com/ask/answers/042415/what-difference-between-moral-hazard-and-adverse-selection.asp
https://www.investopedia.com/articles/pf/07/risk_tolerance.asp
None
https://www.investopedia.com/terms/u/ustreasury.asp
https://www.investopedia.com/terms/d/derivative.asp
https://www.investopedia.com/terms/o/option.asp
https://www.investopedia.com/terms/f/futures.asp
https://www.investopedia.com/terms/s/subprime-meltdown.asp
https://www.investopedia.com/terms/m/mbs.asp
None
https://www.investopedia.com/terms/b/benchmark.asp
https://www.investopedia.com/terms/v/volatility.asp
None
https://www.investopedia.com/terms/s/standarddeviation.asp
https://www.investopedia.com/terms/d/dispersion.asp
https://www.investopedia.com/terms/a/annualized-total-return.asp
https://www.investopedia.com/terms/s/sp500.asp
None
https://www.investope

In [2]:
class DocumentRanker():
    
    def __init__(self, query, folderPath):
        
        self.documentFolderPath = folderPath
        print("folderPath  : ", folderPath)
        self.query = self.convertQuery(query)
        self.folderDocumentContent = self.loadDocuments()  # Store the content of all the files in dictionary
        self.bm25Model = self.createBM25Model()
    
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
        
    def loadDocuments(self):
        '''
        Load documents from the invest_data folder into a dictionary
        '''
        documentTuple = []
        nounList = self.getNouns()
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        
        for root, dirs, files in os.walk(self.documentFolderPath):
    
            for file in files:
                if file.endswith('txt'):
                    with open(os.path.abspath(os.path.join(root, file)), 'r' , encoding="utf8" ) as f:
                        content = f.read()
                        content = content.lower()
                        doc = nlp(content)
                        tokenizedContent = [tokens.text for tokens in doc if tokens.text]         # Change 1 
                        documentTuple.append((file,tokenizedContent))
        
        
        return documentTuple
    
        
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                print(token.text)
                print(token.pos_)
                nounList.append(token.text)
            
        print("noun list : ", nounList)        
        return nounList
    
        
    def createBM25Model(self):
        '''
        Return a created BM25 model.
        '''
        return gensim.summarization.bm25.BM25([x[1] for x in self.folderDocumentContent])
    
    def rankDocuments(self):
        '''
        Rank the documents in the corpus wrt the query. 
        '''
        query = [tokens.text for tokens in self.query if not tokens.is_punct]
        scores = self.bm25Model.get_scores(query)
        documentScores =  [(self.folderDocumentContent[i][0],scores[i]) for i in range(len(scores))]
        return documentScores
    
    def returnTopK(self,k=5):
        '''
        Return top k ranked documents wrt given user query
        '''
        
        scoreDict = self.rankDocuments()
        return sorted(scoreDict , key = lambda x: x[1], reverse = True)[:k]
    
    def returnTopDocuments(self):
        '''
        Return top documents wrt given user query. This uses a different approach from our 
        previous function. We return all documents with score a standard deviation above the
        mean score of our corpus
        '''
        
        scoreTup = self.rankDocuments()
        #scoreDict = dict(scoreTup)
        scores = np.array([x[1] for x in scoreTup])
        #scores = np.array(list(scoreDict.values()))
        meanScore = np.mean(scores)
        sdScore = np.std(scores)
        maxThreshold = meanScore + 2*sdScore
        topScoresDict = [x for x in scoreTup if x[1] > maxThreshold]
        topDocumentScores = sorted(topScoresDict, key = lambda x: x[1], reverse = True)
        
        return topDocumentScores
    
    def returnTopDocumentsData(self):
        '''
        Return data from the top documents retrieved by returnTopDocuments
        '''
        #query = self.query
        topDocumentScores = self.returnTopDocuments()
        print("topDocumentScores  : ", topDocumentScores)
        topDocumentNames = [x[0] for x in topDocumentScores]
        #print(topDocumentNames)
        
        topDocumentText = [x for x in self.folderDocumentContent if x[0] in topDocumentNames]
        return topDocumentText

In [3]:
class PassageRanker():
    
    def __init__(self, query, topDocumentContent):
        
        self.query = self.convertQuery(query)
        self.topDocumentContent = topDocumentContent
        #self.BM25Model = self.createBM25Model()
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
    
    
    def createBM25Model(self,documentContent):
        '''
        Return a created BM25 model.
        '''
        #return gensim.summarization.bm25.BM25([x[1] for x in self.topDocumentContent])
        return gensim.summarization.bm25.BM25(documentContent)
    
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
    def checkNounMatch(self,paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        #nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        stemmer = SnowballStemmer("english")
        paragraphContent = " ".join([stemmer.stem(token) for token in paragraph])
        
        
        for nouns in nounList:
            
            #paragraphDoc = nlp(paragraphContent)
            #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc]
            
            if stemmer.stem(nouns) in paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
                
            
    def returnParagraphList(self,content):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        paragraphList = []
        paragraph = content#[sepCounter:i] # " ".join() <-
        nounList = self.getNouns()
        if self.checkNounMatch(paragraph,nounList):
                #if all(nouns in paragraph for nouns in nounList):
                   
            paragraphList.append(paragraph)
        return paragraphList
        '''
        paragraphList = []
        sepCounter = 0
        nounList = self.getNouns()

        print(nounList)
        for i,term in enumerate(content):
            
            if term == "--------------------------":
                paragraph = content[sepCounter:i] # " ".join() <-
                print("para : ")
                print(paragraph)
                if self.checkNounMatch(paragraph,nounList):
                #if all(nouns in paragraph for nouns in nounList):
                   
                    paragraphList.append(paragraph)
                
                sepCounter = i+1
        return paragraphList
        '''
        
    
    
    def returnParagraphScores(self,bm25Model,query):
        '''
        Return the score of a paragraph given a model.
        '''
        return bm25Model.get_scores(query)
        
    
    def returnTopPassages(self, k=10):
        '''
        Rank each paragraph from the document and return the top 10 passages from the collection of documents
        '''
        query = [tokens.text for tokens in self.query if not tokens.is_punct]
        
        #documentParagraphScores = [(document[0],self.rankPassages(document[1])) for document in topDocumentContent]
        paragraphScoreTup = []
        paragraphLoL = []
        for documents in self.topDocumentContent:
            #print(documents)
            paragraphLoL.extend(self.returnParagraphList(documents[1]))
        
        
        print(len(paragraphLoL))
        #passageBM25Model = self.createBM25Model(paragraphLoL)
        
        #for documents in self.topDocumentContent:
        #paragraphScores = self.returnParagraphScores(passageBM25Model, query)
        for i,paragraph in enumerate(paragraphLoL):
                
            #paragraphScoreTup.append((paragraph,paragraphScores[i]))
            paragraphScoreTup.append((paragraph,1))
        
        print(" length of paragraphScoreTup : ", len(paragraphScoreTup))
        
        self.topParagraphScores = sorted(paragraphScoreTup, key = lambda x: x[1], reverse = True)
        
        return self.topParagraphScores#[:k]

In [4]:
class bertQAModel():
    
    def __init__(self):
        
        self.tokenizerModel,self.bertQAModel = self.__initializeModel()
    
    def __initializeModel(self):
        '''
        Initialize the Beret Tokenizer and the QA model. Note that this is currently compatible with
        transformers module, NOT pytorch/tensorflow
        '''
        with open("./models/bertTokenizer.pkl","rb") as f:
            tokenizerModel = pickle.load(f)
        
        with open("./models/bertQAModel.pkl","rb") as f:
            bertQAModel = pickle.load(f)
        
        return tokenizerModel, bertQAModel
    
    def stringProcess(self,answer):
        answerSplit = answer.split(" ##")
        return "".join(answerSplit)
    
    def predict(self,text,question):
        '''
        Predict the answer given a passage and a question.
        '''
        input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
        print("INPUT_TEXT : ")
        print(input_text)
        input_ids = self.tokenizerModel.encode(input_text)
        #print("TOKENIZED TEXT : ")
        #print(input_ids)
        token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
        #print("TOKEN TYPE IDS : ")
        #print(token_type_ids)
        start_scores, end_scores = self.bertQAModel(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
        
        
        # Normalized scores
        #startScoresVec = ( (startScoresVec - np.mean(startScoresVec))/np.std(startScoresVec))
        #endScoresVec = ( (endScoresVec - np.mean(endScoresVec))/np.std(endScoresVec))
        
        # Normalized Scores
        #startScoresVec = scale(np.reshape(startScoresVec,newshape=(-1,1)))
        #endScoresVec = scale(np.reshape(endScoresVec,newshape=(-1,1)))
        
        # Normalized Scores
        #softmax = torch.nn.Softmax()
        #start_scores = softmax(start_scores)
        #end_scores = softmax(end_scores)
        #endScoresVec = scale(np.reshape(endScoresVec,newshape=(-1,1)))
        
        #startScoresVec = ( (startScoresVec - np.min(startScoresVec))/np.ptp(startScoresVec))
        #endScoresVec = ( (endScoresVec - np.min(endScoresVec))/np.ptp(endScoresVec))
        
        
        all_tokens = self.tokenizerModel.convert_ids_to_tokens(input_ids)
        answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])
        startScoreInd,endScoreInd = torch.argmax(start_scores),torch.argmax(end_scores)
        startScoresVec = start_scores.detach().numpy().flatten()
        endScoresVec = end_scores.detach().numpy().flatten()
        
        startScoreMax, endScoreMax = startScoresVec[startScoreInd] , endScoresVec[endScoreInd]
        #avgScore = float(np.absolute(startScoreMax) + np.absolute(endScoreMax))
        #avgScore = float(startScoreMax + endScoreMax)
        print("ANSWER RETRIEVED : ")
        print(self.stringProcess(answer))
        return (self.stringProcess(answer),startScoreMax)

In [5]:
# C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data
# C:\Users\piyush.kathuria\Desktop\invest-data
path = r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data"

In [35]:
query = "What are two common types of mbss?"

In [36]:
docRanker = DocumentRanker(query,path)
topDocuments = docRanker.returnTopDocumentsData()

folderPath  :  C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data
common
ADJ
types
NOUN
mbss
NOUN
noun list :  ['common', 'types', 'mbss']
topDocumentScores  :  [('mbs8.txt', 27.18925069326761), ('correlationcoefficient2.txt', 14.69516599541684), ('financialasset8.txt', 14.279465651351543), ('normaldistribution1.txt', 13.869377882774158), ('regression6.txt', 13.139476535879528), ('scenario_analysis5.txt', 11.878436523749176), ('what-difference-between-derivatives-and-options9.txt', 11.842945492518437), ('mbs20.txt', 11.258308264533198), ('optionscontract1.txt', 11.078312346770689), ('regression2.txt', 11.019815578199598), ('mbs19.txt', 10.866760332537586), ('certificateofdeposit15.txt', 10.528099526921242), ('security1.txt', 10.339108392102153), ('activerisk16.txt', 10.160073981995781), ('mutualfund9.txt', 10.073669632391162), ('equity9.txt', 10.052890117388632), ('equity9.txt', 10.052890117388632), ('patent4.txt', 10.052890117388632), ('etf5.txt', 9.943945492305081), (

In [37]:
passageRanker = PassageRanker(query , topDocuments)
passageRanking = passageRanker.returnTopPassages()

1
 length of paragraphScoreTup :  1


In [38]:
passageRanking

[(['there',
   'are',
   'two',
   'common',
   'types',
   'of',
   'mbss',
   ':',
   'pass',
   '-',
   'throughs',
   'and',
   'collateralized',
   'mortgage',
   'obligations',
   '(',
   'cmo',
   ')',
   '.'],
  1)]

In [39]:
bertQA = bertQAModel()
answerTuple = []
nlp = spacy.load("en_core_web_sm")
queryDoc = nlp(query)
query = " ".join([tokens.text for tokens in queryDoc if not tokens.is_punct])

for passage,_ in passageRanking[:10] : 
            
    paraText = " ".join(passage)
    answerTuple.append(bertQA.predict(paraText, query))

INPUT_TEXT : 
[CLS] What are two common types of mbss [SEP] there are two common types of mbss : pass - throughs and collateralized mortgage obligations ( cmo ) . [SEP]
ANSWER RETRIEVED : 
pass - throughs and collateralized mortgage obligations ( cmo )


In [40]:
print("ANSWERS : ")
sortedAnswers = sorted(answerTuple, key = lambda x: x[1], reverse = True)

for item in sortedAnswers:
    print(item)

ANSWERS : 
('pass - throughs and collateralized mortgage obligations ( cmo )', 4.8332224)


In [552]:
## Timing Stemmer and Lemmatizer in both spacy and nltk ##

document = "Investors use a variety of tactics to ascertain risk. One of the most commonly used absolute risk metrics is standard deviation, a statistical measure of dispersion around a central tendency. You look at the average return of an investment and then find its average standard deviation over the same time period. Normal distributions (the familiar bell-shaped curve) dictate that the expected return of the investment is likely to be one standard deviation from the average 67% of the time and two standard deviations from the average deviation 95% of the time. This helps investors evaluate risk numerically. If they believe that they can tolerate the risk, financially and emotionally, they invest."

In [119]:
document = " Time      .    Hey"

In [203]:
## 1. Spacy Lemmatizer ##
nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])

In [204]:
from time import time

start = time()
documentDoc = nlp(document)
nounLemma = [tokens.lemma_ for tokens in documentDoc if tokens.text.strip() != '']
end = time()
print(nounLemma)
print("TIME : ",end - start)

['Time', '.', 'hey']
TIME :  0.01604437828063965


In [205]:
len(topDocuments)

33

In [271]:
def convertQuery(query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc

def getNouns(query):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
def checkNounMatch(paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        #nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        stemmer = SnowballStemmer("english")
        #paragraphContent = " ".join(paragraph)
        stemmedPara = [stemmer.stem(token) for token in paragraph]
        #paragraphDoc = nlp(paragraphContent)
        #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc] 
        #nounDoc = nlp(" ".join(nounList))
        #nounsLemma = [tokens.lemma_ for tokens in nounDoc]
        #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc if tokens.pos_ in ["NOUN","ADJ"]]
        for nouns in nounList: #nounsLemma:
            
            #nounLemma = [tokens.lemma_ for tokens in nounDoc]
            print(stemmer.stem(nouns))
            if stemmer.stem(nouns) in stemmedPara:#paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
                
            
def returnParagraphList(content,nounList):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        paragraphList = []
        sepCounter = 0
        #nounList = getNouns()
        
        for i,term in enumerate(content):
            
            if term == "--------------------------":
                paragraph = content[sepCounter:i] # " ".join() <-
                if checkNounMatch(paragraph,nounList):
                #if all(nouns in paragraph for nouns in nounList):
                    paragraphList.append(paragraph)
                
                sepCounter = i+1
        
        return paragraphList

In [272]:
len(topDocuments)

54

In [230]:
query = "What are the drawbacks of standard deviation?"

In [231]:
queryList = convertQuery(query)

NameError: name 'convertQuery' is not defined

In [232]:
queryList

NameError: name 'queryList' is not defined

In [233]:
nounList = getNouns(queryList)

NameError: name 'getNouns' is not defined

In [266]:
nounList

['common', 'types', 'mbs']

In [267]:
for documents in topDocuments :
    
    
    start = time()
    paragraphList = returnParagraphList(documents[1],nounList)
    end = time()
    print("DOCUMENT : ",documents[0], " , TIME TAKEN : ",end - start)

DOCUMENT :  riskmanagement.asp.txt  , TIME TAKEN :  0.04189729690551758
DOCUMENT :  basispoint.asp.txt  , TIME TAKEN :  0.017946243286132812
DOCUMENT :  derivative.asp.txt  , TIME TAKEN :  0.061411142349243164
DOCUMENT :  equity.asp.txt  , TIME TAKEN :  0.051119089126586914
DOCUMENT :  mbs.asp.txt  , TIME TAKEN :  0.025313138961791992
DOCUMENT :  prospecttheory.asp.txt  , TIME TAKEN :  0.017072677612304688
DOCUMENT :  balancesheet.asp.txt  , TIME TAKEN :  0.03179740905761719
DOCUMENT :  bell-curve.asp.txt  , TIME TAKEN :  0.01796889305114746
DOCUMENT :  benchmarkerror.asp.txt  , TIME TAKEN :  0.009974241256713867
DOCUMENT :  calculating-covariance.asp.txt  , TIME TAKEN :  0.02587723731994629
DOCUMENT :  certificateofdeposit.asp.txt  , TIME TAKEN :  0.1207284927368164
DOCUMENT :  correlationcoefficient.asp.txt  , TIME TAKEN :  0.021047353744506836
DOCUMENT :  derivative.asp.txt  , TIME TAKEN :  0.06382894515991211
DOCUMENT :  equity.asp.txt  , TIME TAKEN :  0.05587577819824219
DOCUMENT 

In [236]:
stemmer = SnowballStemmer("english")

In [268]:
[stemmer.stem(x) for x in nounList]

['common', 'type', 'mbs']

In [275]:
document = "Types of Mortgage - Backed Securities . There are two common types of MBSs : pass - throughs and collateralized mortgage obligations ( CMO ) ."

In [276]:
document.split()

['Types',
 'of',
 'Mortgage',
 '-',
 'Backed',
 'Securities',
 '.',
 'There',
 'are',
 'two',
 'common',
 'types',
 'of',
 'MBSs',
 ':',
 'pass',
 '-',
 'throughs',
 'and',
 'collateralized',
 'mortgage',
 'obligations',
 '(',
 'CMO',
 ')',
 '.']

In [274]:
checkNounMatch(["There","are","two","common","types","of","mbss"],nounList)

common
type
mbs


False

In [367]:
stem = SnowballStemmer('english')

In [579]:
stem.stem('security')

'secur'

In [514]:
nlp = spacy.load("en_core_web_sm")

In [58]:
doc = nlp("What is implied volatility")

In [59]:
[tokens.pos_ for tokens in doc]

['PRON', 'AUX', 'VERB', 'NOUN']

In [585]:
x = "Securities there!!!"

In [586]:
doc = nlp(x)

In [587]:
query = [tokens.lemma_ for tokens in doc]

In [588]:
query

['security', 'there', '!', '!', '!']

TypeError: 'bool' object is not iterable